In [1]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
import datetime as dt
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl

import statsmodels
import scipy.stats as ss
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 경로 직접 지정
font_path = 'C:/Windows/Fonts/malgun.ttf'  # 맑은 고딕의 경로
font_prop = fm.FontProperties(fname=font_path, size=12)

# 전역 폰트 설정
plt.rcParams['font.family'] ='Malgun Gothic'  # 맑은 고딕으로 설정
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [3]:
keys = pd.read_csv('../../cp_data.csv')
keys

,media,id,secret
0,cp_dynamic,dc4d4ead-01fc-40e9-8f72-5facafced571,b4458644291d8d0e2a56d55368b17f6909345ae2
1,cp_sp_click,2f3bc594-6b43-4ebf-ba5b-7ef38e2dd7a4,ac99740e4ae9a1ebf3f352f5b4e082dca15237fe
2,nv_shoppiong,XE90vII3DL80ycgIPRzG,tp6GiGD5_d
3,nv_search,DhLceR4k10A9Mqi3kT4U,61w6F8i7_L


In [4]:
access_key = keys['id'][1]
secret_key = keys['secret'][1]


In [6]:
# report_type = 'cancels'
report_type = 'orders'

In [10]:
import requests 
import hmac
import hashlib
from time import gmtime, strftime
from datetime import datetime, timedelta
import pandas as pd
import time

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = f"/v2/providers/affiliate_open_api/apis/openapi/v1/reports/{report_type}"

def generateHmac(method, url, secretKey, accessKey, queryString=""):
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString
    signature = hmac.new(bytes(secretKey, 'utf-8'), message.encode('utf-8'), hashlib.sha256).hexdigest()
    return f"CEA algorithm=HmacSHA256, access-key={accessKey}, signed-date={datetimeGMT}, signature={signature}"

from tqdm import tqdm  # Import tqdm

def fetch_data(startDate, endDate, page, retries=3):
    queryString = f"startDate={startDate}&endDate={endDate}&page={page}"
    fullURL = f"{DOMAIN}{URL}?{queryString}"
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    headers = {"Authorization": authorization, "Content-Type": "application/json"}
    
    for attempt in range(retries):
        try:
            response = requests.get(fullURL, headers=headers, timeout=30)
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Failed to fetch data: HTTP {response.status_code} - {response.text}")
                time.sleep(2)  # Wait before retrying
                
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(2)  # Wait before retrying
    
    return None

def fetch_data_in_chunks(start_date, end_date):
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    results = []

    date_range = (end - start).days // 30 + 1  # Calculate the number of chunks
    date_chunks = tqdm(range(date_range), desc="Fetching date ranges")

    for _ in date_chunks:
        chunk_end = start + timedelta(days=29)
        if chunk_end > end:
            chunk_end = end

        page = 0
        while True:
            response = fetch_data(start.strftime('%Y%m%d'), chunk_end.strftime('%Y%m%d'), page)
            if response:
                data = response.get('data', [])
                if data:
                    results.extend(data)
                    if len(data) < 1000:
                        break
                    page += 1
                else:
                    break
            else:
                break

        start = chunk_end + timedelta(days=1)
        if start > end:
            break

    return results

# Define your access keys and date range
ACCESS_KEY = access_key
SECRET_KEY = secret_key

start_date = "20240416"
end_date = "20240430"

# Collect all data
all_data = fetch_data_in_chunks(start_date, end_date)
df = pd.DataFrame(all_data)


Fetching date ranges:   0%|          | 0/1 [01:28<?, ?it/s]


In [12]:
# df_1 = df.copy()
# df = pd.concat([df_1,df],axis=0)
df.groupby()

,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
0,20240401,AF0450673,,,312,GOLDBOX,8604948714342258,6317742408,바이오믹스 설탕대신 스테비아 에리스리톨 설탕,1,4800.0,3.0,144.0
1,20240401,AF0450673,,,312,GOLDBOX,8094949414878355,6451137848,베러바이즈 여성 여름 라탄 밀짚 라피아 햇 썬 캡 모자,1,8650.0,3.0,260.0
2,20240401,AF0450673,,,312,GOLDBOX,8964949914767481,6101789528,[로켓프레시] 홍루이젠 호밀빵 햄치즈 샌드위치,6,12560.0,3.0,377.0
3,20240401,AF0450673,,,312,GOLDBOX,8224942615004570,116679203,오뚜기 해물완자 (냉동),1,7980.0,3.0,240.0
4,20240401,AF0450673,,,312,GOLDBOX,8214940814846109,6101789528,[로켓프레시] 홍루이젠 호밀빵 햄치즈 샌드위치,2,5560.0,3.0,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49516,20240430,AF0450673,,,312,GOLDBOX,8334947917220861,5836662870,이지앤프리 프리미엄 아이스볼 얼음트레이 6구 2종 세트,1,9900.0,3.0,297.0
49517,20240430,AF0450673,,,312,GOLDBOX,8364941574833037,7312577080,피죤 바이올렛 가든 섬유유연제 본품,1,16900.0,3.0,507.0
49518,20240430,AF0450673,,,312,GOLDBOX,8314942217071994,7942109300,용돈 박스 봉투 어버이날 현금 이벤트 세금 체납 고지서,2,13100.0,3.0,393.0
49519,20240430,AF0450673,,,312,GOLDBOX,8834949017832119,5479650,곰표 밀가루 중력다목적용,1,3520.0,3.0,106.0


In [32]:
# start_date = "20240501"
# end_date = "202440531"

In [33]:
df.to_parquet(f'../result/cp2_{report_type}_{start_date}_{end_date}.csv')

In [24]:
pd.read_parquet('../result/cp2_20240511_20240520.csv')

,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
0,20240511,AF0450673,,,312,GOLDBOX,8604948779486916,5711322392,청정원 AIDA 스파게티면,5,7251.0,3.0,218.0
1,20240511,AF0450673,,,312,GOLDBOX,8234941479719708,7066609232,동서 맥심 화이트골드 커피믹스,1,20280.0,3.0,609.0
2,20240511,AF0450673,,,311,396399,8364941679350440,7811866054,면사랑 직화 유니짜장 (냉동),1,12570.0,3.0,378.0
3,20240511,AF0450673,,,312,GOLDBOX,8224942579786530,7623115961,[로켓프레시] 국내산 당근 채,1,3490.0,3.0,105.0
4,20240511,AF0450673,,,312,GOLDBOX,8644944779895881,7707429971,더담은 든든한 밥상 전라미,1,25900.0,3.0,777.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26335,20240520,AF0450673,,,312,GOLDBOX,8914945572634421,1232707863,곰곰 호주산 소고기 치마살 구이용 (냉장),2,19200.0,3.0,576.0
26336,20240520,AF0450673,,,312,GOLDBOX,8674940678794361,6431801207,더리빙 선물용 하트플라워 종이가방 10p,1,9200.0,3.0,276.0
26337,20240520,AF0450673,,,960,MLGWGOLDBOX,8874947772794318,7851861917,광천김 밥도둑 명란맛김,1,5200.0,3.0,156.0
26338,20240520,AF0450673,,,312,GOLDBOX,8814944672523329,7006313988,풀무원 특등급 국산콩 네컵연두부,1,2780.0,3.0,84.0
